In [138]:
import itertools as it
import pandas as pd
import numpy as np
import Bio.Seq
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics

## Preparing train data

In [139]:
letters = [("A"), ("T"), ("C"), ("G")]

In [140]:
all_seqs = list(it.product(letters, repeat=4))
all_seqs[0]

('A', 'A', 'A', 'A')

In [141]:
all_seqs_str = []

for elem in all_seqs:
    elem = "".join(elem)
    all_seqs_str.append(elem)

In [142]:
all_seqs_str[0]

'AAAA'

In [143]:
Bio.Seq.reverse_complement(all_seqs_str[0])

'TTTT'

In [144]:
len(all_seqs_str)

256

In [145]:
# removing reverse complements
selected_seq_string = []

for elem in all_seqs_str:
    if Bio.Seq.reverse_complement(elem) not in selected_seq_string:
        selected_seq_string.append(elem)

In [146]:
selected_seq_string = set(selected_seq_string)

In [147]:
len(selected_seq_string)

136

In [148]:
with open('vista1500.txt', 'r') as f:
    vista1500 = f.readlines()

with open('randoms1500.txt', 'r') as f:
    randoms1500 = f.readlines()

In [149]:
vista_seqs = []
random_seqs = []

for elem in vista1500:
    if elem[0] != ">":
        vista_seqs.append(elem[:-1])

for elem in randoms1500:
    if elem[0] != ">":
        random_seqs.append(elem[:-1])        

In [150]:
vista_seqs[0]

'GAGGTCAGATAtggcacagtgaagagcaacgtcttctcaatcaagcagggctgggttcaagtctcagctctacctctgactggggaacctggagcagtgagtcaagctctcctaacctcagtttcctcttttgtaaaataaagtagatagaaccactttacagggttattggaagagctaaaagagatgttgcataaaatcatcttgaacagtgtctCTTCAAGCTTGGAAAGGTGATAAAAATTAATGTTTTCCTAAATTCTCCTTCCTTGGTTAACCACTTGGCTATGAAGATTTTATTGGCTGGGATGAAGAAGGAGGACTAAGAAAAGAAAAGTCAGAGAGAGGAGAACAGGAGGAAGATAAAATGTAGacacacgctcacatgcacacacacaaagacacacatacacacacacaGGCAAAGGCCAACTGAAGGGACCCCGTTAGCATATAAACAAAAGGTGGGGGGTAGCCCCGAGCCTCTTCTCTGACAGCCAGTGGCGGCAGTGATGAATTTGTGAAGTTATCTAATTTTCCACTGTTTTAATTAGAGACTTGGGCTCTGAGGCCTCGCAGCTGGCTTCTTTGTGCTGTATTCTGTTGCCTGACAGAGAAAAATGTCTCCTGTAACGTCAGCCAAGCTCTCCGCCAGACCTGAGCAAGCGAAACTTCTGGGATTCATAAACTTGTGGTTTCTGGGTAGAGTGGCGTTTAAACCAGGACTCAGTGGGGAAAGGGCAACATGGCCAGCTCTTCTCCCCAGCGAATCCTCGGAACCAAGGTTGGGGTCCACCATCATCGAAGGGGTGCTGCGGAAAAGGCACGGCCCAGAAAGCCCCCTGAGGATTGTTCTGGGGGTCCTTGATCCTAGTCCATGTGAAATGGAGTCTCCTTGTGGCATGTAATTGAGCCCAGCTTAGAAAGGCCAGTGCTCTGCTTCTCCGAGACAGTGCCTTTGATTGCAGAGTGTGTGATCTGAGTAATTTAATTTATCGC

In [151]:
vista_seqs[0][0:4]

'GAGG'

In [152]:
def count_sequences(dna_seq):
    
    counts = {el:0 for el in selected_seq_string}
    
    for i in range(len(dna_seq)):
        elem = dna_seq.upper()
        window = elem[i:i+4]
        if len(window)>3:
            #print(window)
            if window in selected_seq_string:
                counts[window] += 1
            else:
                counts[Bio.Seq.reverse_complement(window)] += 1
                
    counts = {k: v / 1500 for k, v in counts.items()}
    
    return counts

In [153]:
counts_true_list = []
counts_false_list = []

for elem in vista_seqs:
    counts_true_list.append(count_sequences(elem))
    
for elem in random_seqs:
    counts_false_list.append(count_sequences(elem))

In [154]:
train_true_df = pd.DataFrame.from_dict(counts_true_list)
train_true_df['y'] = pd.Series(np.ones(len(train_true_df)), index=train_true_df.index)

In [155]:
train_false_df = pd.DataFrame.from_dict(counts_false_list)
train_false_df['y'] = pd.Series(np.zeros(len(train_false_df)), index=train_false_df.index)

In [156]:
train_df = pd.concat([train_false_df, train_true_df])

In [157]:
train_df.shape

(2335, 137)

In [158]:
train_df.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTAG,TTCA,TTCC,TTCG,TTGA,TTGC,TTGG,TTTC,TTTG,y
0,0.031333,0.011333,0.022667,0.024000,0.012667,0.007333,0.002667,0.012667,0.020000,0.008667,...,0.004667,0.022000,0.012667,0.002000,0.015333,0.008667,0.004667,0.023333,0.018667,0.0
1,0.028667,0.010667,0.020000,0.023333,0.008000,0.008000,0.000000,0.014000,0.022000,0.007333,...,0.007333,0.010667,0.018667,0.000000,0.012667,0.009333,0.010000,0.024000,0.012667,0.0
2,0.024000,0.013333,0.014667,0.013333,0.012000,0.006667,0.000000,0.006000,0.015333,0.012000,...,0.006667,0.014000,0.015333,0.000000,0.007333,0.010000,0.012000,0.016000,0.014000,0.0
3,0.050000,0.009333,0.012000,0.029333,0.016667,0.006000,0.000667,0.010000,0.014000,0.004667,...,0.011333,0.012667,0.007333,0.002000,0.014667,0.008667,0.009333,0.012667,0.018000,0.0
4,0.010000,0.001333,0.010667,0.005333,0.003333,0.002667,0.001333,0.002667,0.012000,0.004667,...,0.002667,0.001333,0.004667,0.001333,0.006000,0.006000,0.004000,0.007333,0.006667,0.0


## Preparing test data

In [159]:
with open('chr21.fa', 'r') as f:
    chr21 = f.readlines()

In [160]:
chr21_seq = ''.join(chr21[1:]).replace('\n','')

In [161]:
len(chr21_seq)

46709983

In [162]:
chr21_record_list = []

for i in range(0,len(chr21_seq),750):
    window_1 = chr21_seq[i:i+1500]
    
    if i%1000000 == 0:
        print(i)
    
    if len(window_1) > 1499:
        if 'N' not in window_1 and 'n' not in window_1:
            chr21_record_list.append(count_sequences(window_1))
        else:
            chr21_record_list.append({el:0 for el in selected_seq_string})
    
    #if len(window_2) > 1499 and 'N' not in window_2 and 'n' not in window_2:
    #    chr21_record_list.append(count_sequences(window_2))

0
3000000
6000000
9000000
12000000
15000000
18000000
21000000
24000000
27000000
30000000
33000000
36000000
39000000
42000000
45000000


In [163]:
test_df = pd.DataFrame.from_dict(chr21_record_list) 

In [164]:
test_df.shape

(62278, 136)

In [165]:
test_df['has_N'] = False

In [166]:
test_df.loc[test_df.eq(0).all(1),'has_N'] = True

In [167]:
test_df.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTAG,TTCA,TTCC,TTCG,TTGA,TTGC,TTGG,TTTC,TTTG,has_N
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [168]:
test_df.shape

(62278, 137)

## Building model

In [169]:
y = train_df.loc[:,'y']
X = train_df.loc[:, train_df.columns != 'y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [170]:
clf = RandomForestClassifier(n_estimators=250, max_depth=5, random_state=0)
distributions = dict(n_estimators=[100,150,200,250,300,400,500], max_depth=[2,3,4,5])
#clf.fit(X_train, y_train)

In [171]:
clf = RandomizedSearchCV(clf, distributions, random_state=0)
search = clf.fit(X_train, y_train)
search.best_params_

C:\Users\kspalinska\Anaconda3\envs\py367\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 250, 'max_depth': 5}

In [172]:
y_pred_proba = clf.predict_proba(X_test)
metrics.roc_auc_score(y_test, y_pred_proba[:,1])

0.8663264486056418

In [173]:
y_pred = clf.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

0.7224383916990921

## Calculate predicions

In [176]:
clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(X, y)
y_pred = clf.predict_proba(test_df.loc[test_df['has_N']==False, test_df.columns != 'has_N'])

C:\Users\kspalinska\Anaconda3\envs\py367\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [177]:
# mean prediction value for whole chromosome
y_pred[:,1].mean()

0.6821448609716236

In [180]:
len(y_pred[:,1])

53355

In [185]:
test_df[test_df['has_N']==False].shape

(53355, 137)

In [203]:
test_df.loc[test_df['has_N']==False,'prediction'] = y_pred[:,1]

In [204]:
test_df.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTCA,TTCC,TTCG,TTGA,TTGC,TTGG,TTTC,TTTG,has_N,prediction
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145


In [205]:
test_df.loc[test_df['has_N']==True, 'prediction'] = y_pred[:,1].mean()

In [206]:
test_df.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TTCA,TTCC,TTCG,TTGA,TTGC,TTGG,TTTC,TTTG,has_N,prediction
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.682145


## Prepare results in WIG format

In [200]:
test_df['prediction'].values

array([0.68214486, 0.68214486, 0.68214486, ..., 0.68214486, 0.68214486,
       0.68214486])

In [211]:
with open("final_predictions.txt", "w") as f:
    f.write("fixedStep chrom=chr21 start=0 step=750\n")

In [212]:
with open("final_predictions.txt", "ab") as f:
    np.savetxt(f, test_df['prediction'].values, fmt='%4.6f', delimiter='\n')